# Import Libraries

In [3]:
!pip install owlready2
!pip install anytree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 27.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.44-cp310-cp310-linux_x86_64.whl size=24075246 sha256=11a61ebd900e18b61e24abf4bab15689c0ec6436e5b3de5fb5c31fd9b48a2b92
  Stored in directory: /root/.cache/pip/wheels/88/e1/04/583b0743b2907f091204baaa0aef9740f5ba5f3d2f6a5aa00d
Successfully built owlready2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.3 MB/s eta 0:00:00


In [4]:
import plotly.express as px
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
import pickle
import os

from owlready2 import set_render_func
from owlready2 import get_ontology
from owlready2 import default_world

from anytree import AnyNode, Node, RenderTree
from anytree import search
from anytree.search import find
from anytree import PostOrderIter

from operator import add


In [5]:
#Connect to drive to import files
from google.colab import drive
drive.mount('/content/drive')
#drive.mount('/gdrive', force_remount=True)

Mounted at /content/drive


# Import Volumetric File

In [8]:
#Put path to your file with volumetric information here.
#We have 9 columns of meta data information here, including eTIV is a measuemenent of total intracranial volume.

#There are 100 ROIs. FastSurfer produces 104 volumetric segments.
#We club four ROI's left and right regions under one child-ROI, namely the 3rd-Ventricle, 4th-Ventricle, CSF, and brain stem.
#Hence the resultant 100 ROIs.

file_df = pd.read_csv( '/content/drive/My Drive/ontology/FastSurfer_Volumes_combined.csv', index_col=False)

#For example, if we only want to plot samples from a given cohort
cohort = 'ADNI3'
df = file_df[file_df['sample']==cohort]

# or if we only want to plot a certain disease stage
# stage = 'AD'
# cohort = 'All'
# df = file_df[file_df['grp']==stage]

#Consdering all the samples used in our study
# df = file_df[file_df['grp'].isin(['CN','MCI','AD'])]

In [9]:
print('number of data samples:', len(df))
df.head()

number of data samples: 572


,sid,sample,grp,sex1f,age,education_y,MMSE,MRI_field_strength,eTIV,3rd-Ventricle,...,Right-Cerebral-White-Matter,Right-choroid-plexus,Right-Hippocampus,Right-Inf-Lat-Vent,Right-Lateral-Ventricle,Right-Pallidum,Right-Putamen,Right-Thalamus-Proper,Right-VentralDC,WM-hypointensities
952,6001,ADNI3,CN,1.0,72.0,16.0,29,3.0,1209905,1853.9,...,174640.5,662.7,3090.1,370.1,15634.7,1550.9,3758.7,5037.0,3101.0,1853.1
953,6002,ADNI3,MCI,0.0,72.0,16.0,30,3.0,1393639,1181.7,...,194609.9,930.7,4328.4,342.2,9956.4,1823.5,4207.2,5366.7,3885.1,1281.2
954,6005,ADNI3,CN,1.0,67.0,16.0,29,3.0,1520953,1479.5,...,244781.9,713.6,4248.9,466.9,9851.1,1888.4,3251.3,6562.4,4037.2,3462.0
955,6007,ADNI3,CN,1.0,77.0,13.0,29,3.0,1462717,1623.8,...,232120.8,975.2,3908.6,303.4,10426.1,2011.4,4292.4,7167.4,3831.5,4764.6
956,6008,ADNI3,CN,1.0,63.0,16.0,29,3.0,1388685,1031.5,...,196437.7,942.3,3934.3,404.4,11055.9,1880.0,4231.6,6680.5,4217.9,13050.0


# Import onto, use parent-child relationships to find parent ROI volumes

In [14]:
#Get the OWL/Protege ontology which encodes neuroanatomical structral information
onto = get_ontology("/content/drive/My Drive/ontology/tree-structure-brain-anatomy-FastSurfer-v2.rdf").load()


# Create an empty tree,
# to later represent the parent-child relationships amongst ROIs and easily traverse amongst them
root = Node("root", ontoClass=onto.Brain_Region)


#For pretty printing of the entitiy names
def render_using_label(entity):
    return entity.label.first() or entity.name
set_render_func(render_using_label)

##preprocess

In [15]:
def parents_pair_func():
  #Find the pairs of parent-child relationships

  # query all parent classes with root class Brain_Region in ontology
  # request only direct children, not grand(grand)children pairs
  parents_pairs = list(default_world.sparql("""
  SELECT ?toplevel ?leaf
      WHERE {
          ?leaf rdfs:subClassOf ?toplevel;
                rdfs:subClassOf* tree-structure-brain-anatomy-FastSurfer-v2:Brain_Region.
  }
  """)) # prefixes are automatically defined, tree-structure-brain-anatomy-FastSurfer: is the currently loaded ontology

  print('parents_pairs')
  print(len(parents_pairs), parents_pairs)
  return parents_pairs


In [16]:
def add_children_to_tree(root):
  parents = list()
  class_map = dict()
  parents_pairs = parents_pair_func()

  # first: add children and parents to the map
  for pp in parents_pairs:
    if pp[0] not in class_map:
      class_map[pp[0]] = [pp[1]] # add new list with child as first entry
    else:
      class_map[pp[0]].append(pp[1]) # add child class to existing list in map
    if pp[0] not in parents: parents.append(pp[0])

  # second: recursively add children and parents to the tree
  def add_children(p, vals):
    for v in vals:
      pp = AnyNode(parent=p, ontoClass=v) # create child nodes for each value
      add_children(pp, class_map.get(v, [])) # recursively process children

  add_children(root, class_map[onto.Brain_Region])
  print('parents')
  print(len(parents), parents)

  return root, parents_pairs, parents

In [17]:
root, _, parents = add_children_to_tree(root)

parents_pairs
123 [[Corpus_Callosum, Anterior], [White_Matter, Corpus_Callosum], [Brain_Region, Brainstem], [Cerebrospinal_Fluid, CSF], [Brain_Region, Cerebrospinal_Fluid], [Corpus_Callosum, Central], [Brain_Region, Cerebellum], [Brain_Region, Cerebrum], [Cerebrum, White_Matter], [Left_Temporal_Lobe, Left_Amygdala], [Left_Cerebrum, Left_Temporal_Lobe], [Left_Cerebrum, Left_Basal_Ganglia], [Cerebrum, Left_Cerebrum], [Left_Cingulate_Gyrus, Left_Caudal_Anterior], [Left_Cerebrum, Left_Cingulate_Gyrus], [Left_Frontal_Lobe, Left_Caudal_Middle_Frontal], [Left_Cerebrum, Left_Frontal_Lobe], [Left_Basal_Ganglia, Left_Caudate_Nucleus], [Cerebellum, Left_Cerebellum], [Cerebellum, Right_Cerebellum], [Left_Cerebellum, Left_Cerebellum_Cortex], [Left_Cerebellum, Left_Cerebellum_White_Matter], [White_Matter, Left_Cerebral_White_Matter], [Cerebrospinal_Fluid, Left_Choroid_Plexus], [Left_Occipital_Lobe, Left_Cuneus], [Left_Cerebrum, Left_Occipital_Lobe], [Left_Cerebrum, Left_Diencephalon], [Left_Temporal

In [18]:
#print(RenderTree(root).by_attr('ontoClass'))

In [19]:
def child_leaf_names(onto):
  #Find the names of all the chilren/leaf nodes
  children_name, children_id = [], []
  for region in onto.Brain_Region.descendants(include_self=False):
      with onto:
        if len(region.descendants(include_self=False))==0:  # leaf node
          assert region.ID, f"region ID not set for {region}"
          #children.append(str(region.name))
          children_name.append(str(region.name))
          children_id.append(region.ID[0])

  return children_name, children_id


children_name, children_id = child_leaf_names(onto)
children_dict_id_to_name = dict(zip(children_id, children_name))
#children_dict_name_to_id = dict(zip(children_name, children_id))

In [ ]:
def find_parent_volume(df, parents, root):
  '''
  Additively finds true/ground_truth volumetric measurements for all parent ROIs

  df: dataframe with children ROI volumes. The ROI names are still 'id' format.
  parents: str list, a list of all the parent ROI names.
  root: anytree root node, represnting the whole ontology as a tree.
  '''
  for parent in parents:
    vol=0
    #finds all the leaf children (or child-ROI) for any give parent node
    relevant_leaf_children = search.find(root, lambda node: node.ontoClass.name == str(parent)).leaves
    for ele in relevant_leaf_children:
      vol += df[str(ele.ontoClass.ID[0])]

    df[str(parent)] = vol
  return df

df = find_parent_volume(df, parents, root)

In [21]:
#Renaming children ROI columns in the dataframe (from id -> readable name)
df.rename(columns=children_dict_id_to_name, inplace=True)

<ipython-input-21-ac53b3731486>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns=children_dict_id_to_name, inplace=True)


# Import Learned LR models

In [24]:
#Read a dictionary contianing the model coeficents and intercepts for all ROIs.
#This dict also contains std deviation of the control sample's residuals,
#for easy calculation of the w-score

with open('/content/drive/My Drive/ontology/ROI_model_dict.pkl', 'rb') as f:
    ROI_model_dict = pickle.load(f)

#for example, the LR model for 'Right_Hippocampus' is:
ROI_model_dict['Right_Hippocampus']

{'model_coef': array([-6.26111814e+01, -2.62544904e+01,  1.53730465e+01,  1.08188636e-03]),
 'model_intercept': 4172.174640130828,
 'std_residual': 350.10038014610996}

# Find w-scores

In [25]:
def calc_w_batch(col_measured_vol=0.0, col_covariate_values=None, ROI=None, ROI_model_dict=None):
  '''
  A function to calculate w-scores for any ROI, for a given sample.

  col_measured_vol:   the measured/ground_truth ROI volumes for a given region,
  col_covariate_values: (array) list of covariate information assocaited with a given sample,
                      used for predicting expected ROI vol levels
  ROI: (str) name of the ROI,
  ROI_model_dict: (dict) a object containing all the learned LR models with
                  thier coefficients, intercept. Also contain the
                  std. dev. of the control sample's residuals.

  '''
  #Empty model instance
  model = LinearRegression()
  #setting trained weights (model's coef and intercept)
  model.coef_ = ROI_model_dict[ROI]['model_coef']
  model.intercept_ = ROI_model_dict[ROI]['model_intercept']
  std_dev = ROI_model_dict[ROI]['std_residual']

  sample_x = np.array(col_covariate_values)

  w = (col_measured_vol - model.predict(sample_x)) / std_dev

  return w

In [26]:
list_ROIs = children_name + parents

In [27]:
#a new dataframe for storing w-scores
df_w = df.copy(deep=True)

In [28]:
#Find the w-score for the ROIs
for ROI in list_ROIs:
  ROI = str(ROI)
  df_w[ROI] = calc_w_batch(col_measured_vol= df[ROI].values,
                                col_covariate_values = df[['sex1f','age','MRI_field_strength','eTIV']].values,
                                ROI=ROI,
                                ROI_model_dict=ROI_model_dict
                                )

# Plot and Save

In [29]:
# Dataframe with which scores z-scores/w-scores for an AD-sample; To be used later, as it has the parent-child relations built into it.
# And the ROIs are manually ordered and have an associated (rign) 'level' with them,
df_ = pd.read_csv('/content/drive/My Drive/ontology/vis_test.csv',index_col=False).fillna('')
df_.drop(['cn_vol', df_.columns[0]], axis=1, inplace=True)  #dropping excessive columns
df_.rename(columns={"z": "w-score"}, inplace=True)

In [30]:
def sort_code (df):
  #To create symertirc Left and Right cerebrum regions, we sort them based on hand-made codes.
  #In this function, we match each region to their sorting codes.

  df_temp = pd.read_csv('/content/drive/My Drive/ontology/roi_SortSymmetryCode.csv')
  sort_dict = dict(zip(df_temp.ROI, df_temp.sort_try))

  df['sort'] = 0
  for i in df.index:
    df.loc[i, 'sort'] = sort_dict[str(df.loc[i, 'ROI'])]

  return df

In [31]:
#A custom colour scale
custom_RYR = [
    "rgb(81,0,17)",  #~rosewood dark red

    "rgb(165,0,38)",  #shades of red
    "rgb(215,48,39)",
    "rgb(244,109,67)",
    "rgb(253,174,97)",

    "rgb(254,235,181)",  #Yellow
    "rgb(250,248,233)", #lighter yellow
    "rgb(254,235,181)",  #Yellow

    "rgb(253,174,97)", #shades of red, reversed
    "rgb(244,109,67)",
    "rgb(215,48,39)",
    "rgb(165,0,38)",

    "rgb(81,0,17)"  #~rosewood dark red
      ]

##Ex1: Plot a mean w-score plot for any disease stage

In [33]:
#collect all ROI w-scores for a disease stage and average them
stage = 'AD'
# cohort= 'All'
df_stage_w = df_w[df_w['grp']==stage]
# df_stage_w = df_w
print(df_stage_w.shape)

w_dict = df_stage_w[df_stage_w.columns[9:]].mean().to_dict()

#copy the sample-df with the parent-child relations, and replace the w-scores
df_plot = df_.copy()
for index in df_plot.index:
  df_plot.loc[index, 'w-score'] = w_dict[df_plot.loc[index].ROI]
df_plot['w-score'] = df_plot['w-score'].round(2)   #round up the w-scores to get rid of the trailing digits

#cleaning up the names of the ROIs
df_plot['ROI'] = df_plot['ROI'].map(lambda x: x.replace('-',' ').replace('_',' '))
df_plot['parent'] = df_plot['parent'].map(lambda x: x.replace('-',' ').replace('_',' '))

#Add the sorting code to the dataframe and sort the w-scores based on it
df_plot = sort_code(df_plot)
df_plot.sort_values('sort', inplace=True)


(62, 133)


In [34]:
fig1 = px.sunburst(df_plot, names='ROI', parents='parent',
                   color='w-score',
                   color_continuous_scale=custom_RYR,
                   color_continuous_midpoint= 0,
                   range_color=[-5,5],
                   maxdepth = 5,      #controls the number of levels one sees at a time, choose between [4,5]
                   )
fig1.update_layout(title_text="Sunburst ROI Volume W-score Diagram | Mean {} Distribution | Data Cohort: {}".format(stage, cohort),
                   font_size=10,
                   autosize=False,  width=800, height=800)

#Turn off plotly's internal sorting, and rotate the chart to get the vertical L-R alignment
fig1.update_traces(sort=False, rotation=-101, selector=dict(type='sunburst'))    #100,53


fig1.show()
fig1.write_html("{}_{}_mean_w_Sunburst.html".format(cohort,stage))

##Ex2: Plot a single sample's w-score plot

In [35]:
#If we pick a random AD sample for plotting.
sample_id = ['6650']
stage = 'AD'
cohort='ADNI3'
df_sample_w = df_w[df_w['sid'].isin(sample_id)]

w_dict = df_sample_w[df_sample_w.columns[9:]].mean().to_dict()

#copy the sample-df with the parent-child relations, and replace the w-scores
df_plot = df_.copy()
for index in df_plot.index:
  df_plot.loc[index, 'w-score'] = w_dict[df_plot.loc[index].ROI]
df_plot['w-score'] = df_plot['w-score'].round(2)   #round up the w-scores to get rid of the trailing digits

#cleaning up the names of the ROIs
df_plot['ROI'] = df_plot['ROI'].map(lambda x: x.replace('-',' ').replace('_',' '))
df_plot['parent'] = df_plot['parent'].map(lambda x: x.replace('-',' ').replace('_',' '))

#Add the sorting code to the dataframe and sort the w-scores based on it
df_plot = sort_code(df_plot)
df_plot.sort_values('sort', inplace=True)

In [36]:
fig2 = px.sunburst(df_plot, names='ROI', parents='parent',
                   color='w-score',
                   color_continuous_scale=custom_RYR,
                   color_continuous_midpoint= 0,
                   range_color=[-5,5],
                   maxdepth = 5,      #controls the number of levels one sees at a time, choose between [4,5]
                   )
fig2.update_layout(title_text="Sunburst ROI Volume W-score Diagram | Sample ID: {} | Data Cohort: {}".format(sample_id[0], cohort),
                   font_size=10,
                   autosize=False,  width=800, height=800)

#Turn off plotly's internal sorting, and rotate the chart to get the vertical L-R alignment
fig2.update_traces(sort=False, rotation=-101, selector=dict(type='sunburst'))    #100,53


fig2.show()
#fig2.write_image("{}_{}_mean_w_Sunburst.html".format(sample_id[0],cohort))
fig2.write_html("{}_{}_mean_w_Sunburst.html".format(sample_id[0],cohort))